# textrank = technology

In [ ]:
!pip install summa

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for summa: filename=summa-1.2.0-py3-none-any.whl size=54386 sha256=80fbb1439ab3e4a2dbc6a39b4ccaf8864ad3651087f813d9c62489b1599841e0
  Stored in directory: /root/.cache/pip/wheels/4a/ca/c5/4958614cfba88ed6ceb7cb5a849f9f89f9ac49971616bc919f
Successfully built summa


In [ ]:
import requests
import json
import datetime as dt
import pandas as pd

## 크롤링 함수

In [ ]:
def guardian_search(my_api_key, section, date):
  url = "https://content.guardianapis.com/" + section
  final_url = url + "?from-date=" + date + "&to-date=" + date +"&show-fields=bodyText&page-size=200&api-key=" + my_api_key
  response = requests.get(final_url)
  news_data_json = response.content.decode("utf-8")

  # results 부분만 추출
  news_data_crawl = json.loads(news_data_json)
  news_data_crawl_results = news_data_crawl["response"]["results"]

  # results 에서 원하는 부분만 리스트로 추출
  result_list = []
  for search_crawl_context in news_data_crawl_results:
    result_dict = {}
    # result_dict['sectionId'] = search_crawl_context["sectionId"]
    result_dict['article_public_Date'] = search_crawl_context["webPublicationDate"]
    result_dict['title'] = search_crawl_context["webTitle"]
    result_dict['content_url'] = search_crawl_context["webUrl"]
    result_dict['content'] = search_crawl_context["fields"]["bodyText"]
    result_dict["category_id"] = 4
    result_list.append(result_dict)

  result_list

  # df = pd.DataFrame(result_list)

  return result_list

In [ ]:
my_api_key = "6de253c2-9d98-4b21-9750-e4e9b63c8d28"
section = "business"
date = "2024-04-09"

news_list = guardian_search(my_api_key, section, date)

## 딥러닝 모델 요약해서 리턴

In [ ]:
from summa.summarizer import summarize
# summarize(text, ratio=0.2)

In [ ]:
summary_list = []

for item in news_list:
    body_text = item['content']
    summary = summarize(body_text, words=20)
    summary_list.append(summary)

In [ ]:
print(summary_list)

['The FCA said 2.7 million people sought help from a lender, a debt adviser or other financial support charity in the year up to this January.', '', 'Salehpour said he had worked for the company for more than a decade, including on the jets in question, and claimed Boeing employed “shortcuts” in the manufacturing process intended to reduce production bottlenecks – but which ultimately compromised safety.', '', 'M&amp;S said it was setting up a £1m accelerator fund for its ethical project Plan A, working in partnership with longstanding and new suppliers to find novel ways of achieving net zero carbon emissions.', '', 'Shell’s former chief executive has stoked fears that the oil company will quit the London Stock Exchange in favour of a New York listing because US investors are “more positive” about fossil fuels.', 'The Australian investment bank Macquarie, which has been criticised for its role in the privatisation of England’s water industry, is understood be among lenders to Thames W

## 요약한 거 json 파일에 열 추가

In [ ]:
def add_abstracts(news_list, summary_list):

  for i, item in enumerate(news_list):
    # 요약본 추출
    summary = summary_list[i]

    # abstract 열 추가 및 요약본 저장
    item['abstract'] = summary

# abstract 열 추가
add_abstracts(news_list, summary_list)

# 결과 확인 (result_list에는 각 문서의 요약본이 'abstract' 열에 담겨있음)
print(news_list)

[{'sectionId': 'business', 'webPublicationDate': '2024-04-09T23:01:31Z', 'webTitle': 'Paying bills a struggle for 7.4 million UK consumers, regulator finds  ', 'webUrl': 'https://www.theguardian.com/business/2024/apr/10/paying-bills-a-struggle-for-74-million-uk-consumers-regulator-finds', 'bodyText': 'More than 7.4 million people in the UK struggled to pay a bill or a credit repayment in January, according to a financial regulator. The figure is less than last year but is still significantly higher than before the cost of living crisis began. According to the Financial Conduct Authority (FCA), which tracks the number of households in financial difficulties, 5.8 million people reported that they were struggling to pay a large bill in February 2020. The data, which gives an insight into the UK’s personal finances, showed that by January 2023, after the Russian invasion of Ukraine and the subsequent cost of living crisis, the number of people in financial difficulties almost doubled to 10

In [ ]:
import json


with open("/content/drive/MyDrive/2차 프로젝트/NLP model/data/textrank.json", "w") as f:
    json.dump(news_list, f, indent=4)